# SnapPy and SageMath are friends!

* SnapPy always uses part of Sage, namely its interface to PARI, repackaged into a stand-alone module CyPari:

    - Smith formal form for homology computations.

    - Arbitrary precision arithmetic.

    - Available on PyPI.
    

* When installed into Sage, via:
```
sage -pip install --no-use-wheel snappy
```
it gains additional functionality.  GUI still works:
```
sage -python -m snappy.app
```


* Installed in SageMathCloud by default.
   

## Sage-specific features.

Numeric return types are native Sage types.

In [1]:
import snappy
M = snappy.Manifold('m004')
type(M.volume())

sage.rings.real_mpfr.RealNumber

In [2]:
MH = M.high_precision()
MH.volume()

2.02988321281930725004240510854904057188337861506059958403497821

In [3]:
MH.volume().prec() # quad-double

212

In [4]:
M.tetrahedra_shapes('rect', bits_prec=1000)[0]

0.500000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000 + 0.866025403784438646763723170752936183471402626905190314027903489725966508454400018540573093378624287837813070707703351514984972547499476239405827756047186824264046615951152791033987410050542337461632507656171633451661443325336127334460918985613523565830183930794009524993268689929694733825173753288025*I

The quad-double precision kernel allows computing very complicated Dirichlet domains.  



In [7]:
B = snappy.Manifold('13a100')
print(B.volume())
try:
    B.dirichlet_domain()
except RuntimeError, error:
    print(error)

15.2039621209048
The Dirichlet construction failed.


In [8]:
BH = B.high_precision()
BH.dirichlet_domain()

136 finite vertices, 2 ideal vertices; 210 edges; 74 faces

Because of local rigidity, the shapes are always algebraic numbers.  We can use LLL to recover their exact expressions, following Goodman, Neumann, et. al.

In [9]:
TF = M.tetrahedra_field_gens()
TF

<SetOfAAN: [0.5 + 0.8660254037844386*I, 0.5 + 0.8660254037844386*I]>

In [10]:
TF.find_field(100, 10, True)

(Number Field in z with defining polynomial x^2 - x + 1,
 <ApproxAN: 0.5 + 0.866025403784*I>,
 [z, z])

Finite covers are of central interest in 3-manifold theory, e.g. Agol's recent solution to the Virtual Haken Conjecture. SnapPy has some basic ability here:

In [11]:
%time covers = M.covers(9)

CPU times: user 5.22 s, sys: 0 ns, total: 5.22 s
Wall time: 5.22 s


However, in Sage one can use GAP or Magma to explore *much* bigger covers.

In [12]:
from sage.all import gap, magma, PSL
gap(1), magma(1) # one-time startup cost
%time covers = M.covers(9, method='gap')

CPU times: user 340 ms, sys: 76 ms, total: 416 ms
Wall time: 479 ms


In [13]:
%time covers = M.covers(9, method='magma')

CPU times: user 100 ms, sys: 40 ms, total: 140 ms
Wall time: 402 ms


In [14]:
[C.homology() for C in covers]

[Z/21 + Z,
 Z + Z + Z,
 Z + Z + Z,
 Z + Z + Z,
 Z/21 + Z,
 Z/4 + Z/4 + Z,
 Z/3 + Z + Z + Z,
 Z/3 + Z + Z + Z,
 Z/3 + Z + Z + Z,
 Z/3 + Z + Z + Z,
 Z/76 + Z/76 + Z]

In [15]:
G = M.fundamental_group(); G

Generators:
   a,b
Relators:
   aaabABBAb

In [16]:
G_gap = gap(G)
G_gap.RelatorsOfFpGroup()

[ a^3*b*a^-1*b^-2*a^-1*b ]

In [17]:
f = G_gap.GQuotients(PSL(2,7))[1]
C = M.cover(f.Kernel())
C.volume()/M.volume()

167.999999999999

In [18]:
G_mag = magma(G)
H = G_mag.LowIndexSubgroups(12)[174]
N = G_mag.Core(H)
print(G_mag.Index(N))
M.cover(N).homology()

576


Z/2 + Z/4 + Z/4 + Z/4 + Z/4 + Z/20 + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z + Z

## Verified computation

As pioneered by HIKMOT, SnapPy can use interval arithmetic to *prove* that a given manifold is hyperbolic and provide intervals where the exact shapes must lie.  Uses Sage's complex interval types and the Newton interval method.

In [ ]:
E = snappy.Manifold('K14n1234')
success, shapes = E.verify_hyperbolicity()
success, shapes[:5]

In [ ]:
z0 = shapes[0]

In [ ]:
z0.diameter()

In [ ]:
better_shapes = E.verify_hyperbolicity(bits_prec=1000)[1]
max(z.diameter() for z in better_shapes)

By Mostow rigidity, a (finite-volume) hyperbolic structure is unique.  When the manifold has cusps, there is a canonical ideal cell decomposition associated to the hyperbolic structure.  SnapPy uses this to decide when two hyperbolic manifolds are homeomorphic.  

In [ ]:
F = snappy.Manifold('K14n1235')
E.is_isometric_to(F)

That was a numerical calculation and not rigorous because SnapPy might have miscalculated the canonical decompositions.  Let's fix that.

In [ ]:
E.num_tetrahedra(), E.triangulation_isosig()

In [ ]:
E.isometry_signature()

In [ ]:
F.isometry_signature()

## Link Diagrams: Spherogram

Spherogram is a separately installable module, mostly pure-Python, which deals with knot and link diagrams.  Basic data structure is a planar diagram.

In [ ]:
import spherogram
K = spherogram.random_link(300, 1, consistent_twist_regions=True)
K

In [ ]:
K.simplify('global')
K

In [ ]:
K.exterior().volume()

In [ ]:
K.alexander_polynomial()  # 'local' algorithm of Bar-Natan

In [ ]:
L = spherogram.Link('L13n131')

In [ ]:
L.jones_polynomial()

In [ ]:
L.signature()

In [ ]:
L.seifert_matrix()

In [ ]:
D = L.morse_diagram()  # Uses Sage's interface to GLPK

In [ ]:
print(D.is_bridge())
B = D.bridge()
len(B.crossings)

In [ ]:
L.braid_word()

## Spherogram 1.5 and SageMath 7.2 links and braids are friends

In [ ]:
L_sage = L.sage_link()

In [ ]:
L_sage

In [ ]:
L_sage_and_back = spherogram.Link(L_sage)

In [ ]:
L_sage_and_back.exterior().is_isometric_to(L.exterior())

In [ ]:
w = L.braid_word(as_sage_braid=True)
w, w.parent()

In [ ]:
L_braid = spherogram.Link(braid_closure=w)
L_braid

In [ ]:
L_braid.simplify('global')
L

# Other 3-manifold software.

Regina focuses on the purely topological side of things, especially normal surface theory.  Includes some support for higher-dimensional manifolds. Like SnapPy, it has a stand-alone GUI and also a Python interface. 

See http://unhyperbolic.org/sageRegina/ for how to install it painlessly into Sage.

# Possible projects for Sage Days 74

### Make SnapPy (even) easier to install into SageMath

   - Sage optional package?

   - SageMath binaries for OS X < 10.11? 

### Sage's "attach" blocks Tkinter 

In Sage's IPython-based interpreter, the mechanism behind Sage's attach functionality blocks IPython's ability to integrate with Tk (and other GUI's) event loops. (See trac #15152).



### Modernize CyPari

CyPari is based on Sage circa 2012, and there have been improvements since, with more to come (Demeyer et. al.).

### Modularization

Modularize some parts of the SageMath kernel, for example interval arithmetic, for use in stand-alone SnapPy.